In [1]:
import os
import json
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

from sklearn import preprocessing

## DataSet
- [Turbo Fan Engine Degration Simulation Data Set (2008): NASA](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan)
    - Recorded as Multiple MultiVariate Time Series
    - 3  Settings: (setting{x}): operational environment condition values that change over time 
    - 21 Sensors   (s1-s21): Each Sensor measuring info about the physical state of a particular engine
    - Each Engine  (id): Stars with different unknown level of wear and allowed to run until failure
- Terms
    - RUL = Remaining Useful Life (in units of hours or cycles)
    - TUL = Time Until Failure
- Background
    - Sensor Readings from a Fleet of Simulated Gas Turbine Engines
- Goal:
    - Predict when an in-service engine will fail based on target (RUL, Label1, Label2)
    - Determine period ahead to alert before failure is oncoming
- Training Data
    - Sensor measuerments recorded time steps up until failure occurs
    - Every sample in the Training Set will fail => so able to predict TUL value at each time step
    - Number of Sequences per Engine = Number of Cycles for each Engine until Failure
    - Due to their different initial conditions, each engine has a slightly different lifetime and failure pattern
    - Therefore each engine's progress in time is not quite aligned with any other (therefore cannot compare cycles)
    - TUL allows to align different engines data to common endpoint (t=0)
- Testing Data
    - Does not indicate when failure occurs (last time period does not represent failure point)
    - Ground Truth (RUL_FD00x.txt): provides # of remaining work cycles for engines in 'Testing Data' before failure
    - Ground Truth used to generated labels
- Labels:
    - Let w0=15 (last 15 Cycles), w1=30 (last 30 cycles)
    - Regression => per RUL
    - Label1: Binary Classification => fail within (*w1* cycles) certain time frame
    - Label2: Multi-Class Classification => fail within cycles: !within w1 days | *[1,w0]* | *[w0+1, w1]*

### Load/Describe DataSet
- Download [NASA TurboFan Dataset](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan)
- 4 Different Experiments: Load a Particular Single Experiment
- Describe the DataSet: How many unique engines, Are Input Records of Variable Time Series

In [62]:
# Download Dataset
root_dir  = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_root = os.path.join(root_dir, "data")
data_dir  = os.path.join(data_root, "nasa-turbofan")
data_file = os.path.join(data_dir,  "turbofan.zip")
if not os.path.exists(data_dir): 
    os.makedirs(data_dir) 
    !wget  "https://ti.arc.nasa.gov/c/6/" -O "../data/turbofan.zip"
    !unzip "../data/turbofan.zip" -d "../data/nasa-turbofan"
    os.symlink(data_dir, (os.path.join(os.getenv('HOME'), 'Data/nasa-turbofan')), target_is_directory=True) 

In [3]:
# Each row is snapshot of data take during a single operation cycle (1 Cycle=21 Sensor Readings)
def read_data(data_type, data_unit, num_readings=21, drop_cols=None):
    """
    read in the appropriate dataset 
    data_type: {train, test, RUL}
    data_unit: {001-004}
    """
    #format:   one time series per file, separate files for labels 
    #record:   each record is a time step from readings of the sensors for a particular engine id
    #id field: engine id (trajectories)
    #cycle:    as time unit => number of operational cycles since start of engine operation 
    #s1-s21:   each of different sensor units providing readings 
    cols = ['id', 'cycle', 'setting1', 'setting2', 'setting3']
    cols = cols + ["".join(['s',str(el)]) for el in range(1,num_readings+1)]

    df = pd.read_csv(os.path.join(data_dir,"{}_FD{}.txt".format(data_type, data_unit)), header=None, sep=" ")
    df = df.drop(df.columns[drop_cols], axis=1)
    df.columns = cols
    df = df.sort_values(['id','cycle'])
    return df

In [4]:
df_train = read_data('train', '001', num_readings=21, drop_cols=[26, 27])
df_test  = read_data('test',  '001', num_readings=21, drop_cols=[26, 27])
df_rul   = pd.read_csv(os.path.join(data_dir,"RUL_FD001.txt"), header=None, sep =" ").drop([1], axis=1)
# TBD: Look at the different distributions of each dataset partition

In [5]:
print("Partition Shapes: Train: {}, Test: {}, RUL: {}".format(df_train.shape, df_test.shape, df_rul.shape))
print("Number of Engines: {}".format(df_rul.shape[0] + 1))
print("Variable Engine Time Series => ")
display(pd.DataFrame(df_train.groupby('id')['cycle'].count()).T)
df_train[df_train.id == 1].head()

Partition Shapes: Train: (20631, 26), Test: (13096, 26), RUL: (100, 1)
Number of Engines: 101
Variable Engine Time Series => 


id,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
cycle,192,287,179,189,269,188,259,150,201,222,...,135,341,155,258,283,336,202,156,185,200


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


### Prepare Labels
- Need to indicate target for Binary Classification Problem to Solve on Training Data



In [7]:
def calc_maxcycles(df):
    """
    calculates an engines max cycles RUL 
    """
    # acquire maximum cycle until failure of an engine 
    # for each cycle determine delta (RUL) to remaining cycles until failure
    rul = pd.DataFrame(df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    return rul

def update_maxcycles(df_cycles, df_gt):
    """
    used during test dataset to update the RUL value using the GT (df_rul)
    only used for test set, as the number of cycles is not indicative till failure
    """
    df_gt_link = df_gt.copy()
    df_gt_link.columns = ['more']
    df_gt_link['id']   = df_gt_link.index + 1
    df_gt_link['max']  = df_cycles['max'] + df_gt_link['more']
    df_gt_link = df_gt_link.drop('more', axis=1)
    return df_gt_link


def calc_rul(df, df_cycles):
    """
    calculates RUL based on maximum yccles of an engine
    used for regression prediction
    """
    df = df.merge(df_cycles, on=['id'], how='left')
    df['RUL'] = df['max'] - df['cycle']
    df = df.drop('max', axis=1)
    return df
    
def calc_labels(df, w0=15, w1=30):
    """
    calculate labels (binary, multiclass) based on RUL falling within the tail of a cycle window
    """
    df['label1'] = np.where(df['RUL'] <= w1, 1, 0 )
    df['label2'] = df['label1']
    df.loc[df['RUL'] <= w0, 'label2'] = 2
    return df

def scale_features(scaler, df, cols, phase):
    """
    scale certain parameters: cycle_norm
    """
    df['cycle_norm'] = df['cycle']
    cols_normalize = df.columns.difference(cols)
    fn = scaler.fit_transform if phase == 'train' else scaler.transform
    df_norm = pd.DataFrame(fn(df[cols_normalize]), 
                           columns=cols_normalize, 
                           index=df.index)
    df_join = df[df.columns.difference(cols_normalize)].join(df_norm)
    df = df_join.reindex(columns = df.columns).reset_index(drop=True)
    return df

id_cols     = ['id', 'cycle']
target_cols = ['RUL','label1','label2']
scaler_norm = preprocessing.StandardScaler()

df_train_cycles  = calc_maxcycles(df_train)
df_train_rul     = calc_rul(df_train, df_train_cycles)
df_train_labeled = calc_labels(df_train_rul)
df_train_labeled = scale_features(scaler_norm, df_train_labeled, id_cols+target_cols, 'train')

df_test_cycles   = calc_maxcycles(df_test)
df_test_cycles   = update_maxcycles(df_test_cycles, df_rul)
df_test_rul      = calc_rul(df_test, df_test_cycles)
df_test_labeled  = calc_labels(df_test_rul)
df_test_labeled  = scale_features(scaler_norm, df_test_labeled, id_cols+target_cols, 'test')

### Format Export of Dataset
- DataSet should be a Numbered Time Series, with Engine as the Index

In [8]:
def create_path(export_path, partition_dir):
    """
    Create the Partition dir if not existent
    """
   
    path = os.path.join(export_path, partition_dir)
    [ os.makedirs(os.path.join(path, p)) for p in ["features", "labels"]
      if not os.path.exists(os.path.join(path,p))]

def create_sequenced_files(df, data_path, partition_dir):
    """
    Export to CSV Files
    Time Series records per File follows Numerical Inde according to Engine ID
    """
    path = os.path.join(data_path, partition_dir)
    create_path(data_path, partition_dir)
    fname = "turbofan-engine-ts-{}.csv".format(df.id.unique()[0])
    # filter out the output categories from the input categories
    df_features = df.drop(['RUL', 'label1', 'label2'], axis=1)
    df_labels   = df[['id', 'cycle', 'RUL', 'label1', 'label2']]
    df_features.to_csv(os.path.join(os.path.join(path, "features"),fname), index=False)
    df_labels.to_csv(os.path.join(os.path.join(path, "labels"),fname), index=False)

In [10]:
export_data_path  = os.path.join(data_dir, "exports/ckpts/data")
export_model_path = os.path.join(data_dir, "exports/ckpts/models") 
_ = df_train_labeled.groupby('id').apply(lambda fr: 
                                         create_sequenced_files(fr, export_data_path, 'train'))
_ = df_test_labeled.groupby('id').apply(lambda fr:  
                                        create_sequenced_files(fr, export_data_path, 'test'))

### LSTM
- Dimensions [samples, features, timesteps]
  - samples    = num time series (batch_size)
  - timesteps  = sequence length
  - features   = values per timestep (setting{x}, cycle_norm, s1-s21}
- Files
  - Time Series in Single File:    Each record represents time series readings from a single engine
  - Time Series in Multiple Files: Each file represents readings from a single engine
- API
  - Using Functional API rather an Sequential Model (for proper alignment with DL4J Model Import LSTM)


In [11]:
# generate sequences via a lazy batch generator
# function to reshape features into (samples, time steps, features) 
def gen_sequence(df_id, seq_length, seq_cols):
    """ 
    Only sequences that meet the window-length are considered, no padding is used. 
    This means for testing we need to drop those which are below the window-length. 
    An alternative would be to pad sequences so that we can use shorter ones """
    data_array = df_id[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]
        
def batch_gen(df, target, sequence_len):
    """
    batch generator of (features, labels)
    ===> basic method to just use .fit instead of lazy generator
    """
    # filter columns
    sensor_cols = ['s' + str(i) for i in range(1,22)]
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    sequence_cols.extend(sensor_cols)
    # generate sequences and convert to numpy array
    seq_gen = ( list(gen_sequence(df[df['id']==id], sequence_len, sequence_cols)) 
                for id in df_train['id'].unique() )
    seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
    # generate labels and convert to numpy array
    label_gen = [ gen_labels(df[df['id']==id], sequence_len, [target]) 
                  for id in df['id'].unique() ]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return seq_array, label_array
    

# pick the feature columns, set a large window size of 50 cycles 
sequence_length = 50
seq_array, label_array = batch_gen(df_train_labeled, target='label1', sequence_len=sequence_length)
nb_features, nb_out = seq_array.shape[2], label_array.shape[1]
get_activation = lambda nb_neurons: 'softmax' if(nb_neurons > 1) else 'sigmoid'
get_loss       = lambda nb_neurons: 'categorical_crossentropy' if(nb_neurons > 1) else 'binary_crossentropy' 
seq_array.shape, label_array.shape

((15631, 50, 25), (15631, 1))

#### Model Architecture

In [12]:
def lstm_layer(name, units, sequences, shape=None):
    """
    name:  name given to layer
    units: number of input units
    shape: for initial layer input shape (only for first layer)
    sequences: whether to return sequences or not
    """
    data_dict = {
     'name': name,
     'units':units,
     'return_sequences': sequences,
     'kernel_initializer':    'glorot_uniform', 
     'recurrent_initializer': 'glorot_uniform'
    }
    if shape: data_dict.update({'input_shape':shape})
    return LSTM(**data_dict)

In [13]:
### Model the MLP via Sequential API Instead
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, Dropout, Activation

def create_model_cg(sequence_len, num_features, num_classes):
    """
    create a Computational Graph (Functional API)
    sequence_len: length of sequence
    num_features: feature vector shape (columns)
    num_classses: number of classes    (target classifier)
    """
    seq_in = Input(shape=(sequence_len, num_features), name="inputLayer_a")
    lstm_a = lstm_layer(name="lstm_a", units=164, sequences=True)(seq_in)
    dropout_a = Dropout(0.2, name='dropout_a')(lstm_a)

    lstm_b = lstm_layer(name="lstm_b", units=96, sequences=True)(dropout_a)
    dropout_b = Dropout(0.2, name='dropout_b')(lstm_b)
    
    lstm_c = lstm_layer(name="lstm_c", units=48, sequences=False)(dropout_b)
    dropout_c = Dropout(0.2, name='dropout_c')(lstm_c)

    output = Dense(num_classes, activation=get_activation(num_classes), name='outputLayer_a')(dropout_c)
    model   = Model(inputs=seq_in, outputs=output, name="Keras_LSTM")
    return model

def create_model_mlp(sequence_len, num_features, num_classes):
    """
    create a Sequential MLP Model
    sequence_len: length of sequence
    num_features: feature vector shape (columns)
    num_classses: number of classes    (target classifier)
    """
    model = Sequential()
    model.add(lstm_layer(name="lstm_a", units=164, sequences=True, shape=(sequence_len, num_features)))
    model.add(Dropout(0.2, name="dropout_a"))

    model.add(lstm_layer(name="lstm_b", units=96,  sequences=True))
    model.add(Dropout(0.2, name="dropout_b"))

    model.add(lstm_layer(name="lstm_c", units=48,  sequences=False))
    model.add(Dropout(0.2, name="dropout_c"))

    model.add(Dense(units=num_classes, activation=get_activation(num_classes), name="outputLayer_a"))
    return model

model_type = "compgraph"
#model = create_model_mlp(sequence_length, nb_features, nb_out)    
model = create_model_cg(sequence_length, nb_features, nb_out)
model.compile(loss=get_loss(nb_out), optimizer='adam', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputLayer_a (InputLayer)    (None, 50, 25)            0         
_________________________________________________________________
lstm_a (LSTM)                (None, 50, 164)           124640    
_________________________________________________________________
dropout_a (Dropout)          (None, 50, 164)           0         
_________________________________________________________________
lstm_b (LSTM)                (None, 50, 96)            100224    
_________________________________________________________________
dropout_b (Dropout)          (None, 50, 96)            0         
_________________________________________________________________
lstm_c (LSTM)                (None, 48)                27840     
_________________________________________________________________
dropout_c (Dropout)          (None, 48)                0         
__________

#### Model Training + Serialization

In [14]:
%%time
from keras.callbacks import EarlyStopping
# fit the network
# does not shuffle by default
model.fit(seq_array, label_array, epochs=10, batch_size=32, verbose=1,
          validation_split=0.2,
          callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, 
                                     patience=1, verbose=0, mode='auto')])                   

Train on 12504 samples, validate on 3127 samples
Epoch 1/10
12504/12504 [==============================] - 225s - loss: 0.1143 - acc: 0.9516 - val_loss: 0.0707 - val_acc: 0.9722
Epoch 2/10
12504/12504 [==============================] - 217s - loss: 0.0678 - acc: 0.9710 - val_loss: 0.0494 - val_acc: 0.9799
Epoch 3/10
12504/12504 [==============================] - 214s - loss: 0.0593 - acc: 0.9753 - val_loss: 0.0539 - val_acc: 0.9747
Epoch 4/10
12504/12504 [==============================] - 200s - loss: 0.0472 - acc: 0.9787 - val_loss: 0.0534 - val_acc: 0.9767
CPU times: user 22min 34s, sys: 8min 1s, total: 30min 35s
Wall time: 14min 24s


In [16]:
# TBD: Peform evaluate on Test Set

In [21]:
if not os.path.exists(export_model_path): os.makedirs(export_model_path)
serialize_str = lambda path, s, arg: os.path.join(path, s.format(arg))
    
model.save(serialize_str(export_model_path, "model_keras_{}.h5", model_type))
model.save_weights(serialize_str(export_model_path, "model_keras_weights_{}.h5", model_type))
with open(serialize_str(export_model_path,  "model_keras_config_{}.h5", model_type), 'w') as f:
    f.write(model.to_json())
with open(serialize_str(export_model_path,  "layers_keras_config_{}.h5", model_type), 'w') as f:
    json.dumps(model.get_config(), f)                   